<a href="https://colab.research.google.com/github/Amir-Fasil/Prompt_Vs_finetuning/blob/main/Zero_shot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain transformers torch sentence-transformers langchain-community

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 2.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 61.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 42.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 39.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 13.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 101.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 88.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.4/44.4 kB 4.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.

In [ ]:
from langchain.llms import HuggingFacePipeline
from langchain.prompts import PromptTemplate
from langchain.chains import LLMChain
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from google.colab import userdata
from huggingface_hub import login
import os
import torch

**Initialize the Llama 2 model and tokenizer**

In [ ]:
hf_token = userdata.get('HF_TOKEN')
login(token=hf_token, add_to_git_credential=True)

In [ ]:
# Initialize the Llama 2 model and tokenizer
def load_llama2_model():
    model_name = "meta-llama/Llama-2-7b-chat-hf"

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)

    # Load model
    model = AutoModelForCausalLM.from_pretrained(
        model_name,
        device_map="auto",
        torch_dtype=torch.float16,
    )

    # Create text generation pipeline
    pipe = pipeline(
        "text-generation",
        model=model,
        tokenizer=tokenizer,
        max_new_tokens=512,
        temperature=0.7,
        top_p=0.95,
        repetition_penalty=1.15
    )

    # Wrap in LangChain HuggingFacePipeline
    llm = HuggingFacePipeline(pipeline=pipe)

    return llm

llm = load_llama2_model()

**Zero_shot Prompt**

In [ ]:
def create_zero_shot_prompt():

    template = """<s>[INST] <<SYS>>
    You are an AI assistant. Answer the question based on the context below. If you can't answer the question,
    reply 'I don't know' instead of making up an answer.
    <</SYS>>

    {question} [/INST]"""

    return PromptTemplate(
        input_variables=["question"],
        template=template
    )

prompt_template = create_zero_shot_prompt()

**Wrapping prompt and model in a Chain**

In [ ]:
# Create a zero-shot chain
def create_zero_shot_chain(llm, prompt_template):

    return LLMChain(

        llm=llm,
        prompt=prompt_template,
        verbose=True  # Set to True to see the prompt and response details
    )

zero_shot_chain = create_zero_shot_chain(llm, prompt_template)

**Execution**

In [ ]:
def main(zero_shot_chain):

    # Example questions
    questions = [

        "Explain the concept of gravitational waves in simple terms.",
        "What is the capital of France?",
        "How do I bake a chocolate cake without eggs?",
        "What is the meaning of life?",
        "Who won the 2024 Indonesian presidential election?",
        "Which team won Super Bowl LVIII in 2024?",

    ]

    # Get answers
    for question in questions:

        print(f"\nQuestion: {question}")
        response = zero_shot_chain.run(question=question)
        print(f"Answer: {response}")


main(zero_shot_chain)

Loading Llama 2 model...


model.safetensors.index.json:   0%|          | 0.00/26.8k [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

model-00001-of-00002.safetensors:   0%|          | 0.00/9.98G [00:00<?, ?B/s]

model-00002-of-00002.safetensors:   0%|          | 0.00/3.50G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/188 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-8-e18b4156fbd9>:27: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=pipe)
<ipython-input-10-ea993136b2ab>:4: LangChainDeprecationWarning: The class `LLMChain` was deprecated in LangChain 0.1.17 and will be removed in 1.0. Use :meth:`~RunnableSequence, e.g., `prompt | llm`` instead.
  return LLMChain(
<ipython-input-11-f09ac6061b8b>:29: LangChainDeprecationWarning: The method `Chain.run` was deprecated in langchain 0.1.0 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = zero_shot_chain.run(question=question)



Question: Explain the concept of gravitational waves in simple terms.


> Entering new LLMChain chain...
Prompt after formatting:
<s>[INST] <<SYS>>
    You are an AI assistant. Answer the question based on the context below. If you can't answer the question, 
    reply 'I don't know' instead of making up an answer.
    <</SYS>>
    
    Explain the concept of gravitational waves in simple terms. [/INST]

> Finished chain.
Answer: <s>[INST] <<SYS>>
    You are an AI assistant. Answer the question based on the context below. If you can't answer the question, 
    reply 'I don't know' instead of making up an answer.
    <</SYS>>
    
    Explain the concept of gravitational waves in simple terms. [/INST]  Of course! I'd be happy to help explain the concept of gravitational waves in simple terms.

Gravitational waves are ripples in the fabric of spacetime that are caused by massive cosmic events, such as the collision of two black holes or neutron stars. These ripples were predicted by Al